# 메타클래스로 클래스 속성에 주석을 달자

메타 클래스로 구현할 수 잇는 기능 중 하나는 클래스를 정의한 이후에, 하지만 그 클래스를 실제로 사용하기 전에 property를 수정하거나 주석을 붙이는것이다. 보통은 이 기법을 descriptor와 함께 사용하며, 클래스에서 descriptor를 어덯게 사용하는지 자세히 조사한 정보를 descriptor에 제공한다.

In [4]:
class Field(object):
    def __init__(self, name):
        self.name = name
        self.internal_name = '_' + self.name
        
    def __get__(self, instance, instance_type):
        if instance is None: return self
        return getattr(instance, self.internal_name, '')
    
    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)

Field descriptor에 저장할 칼럼 이름이 있으면 내장 함수 setattr과 getattr을 사용해서 모든 인스턴스별 상태를 인스턴스 딕셔너리에 보호 필드로 직접 저장할 수 있다. 처음에는 이 방법이 메모리 누수를 피하려고 weakref로 descriptor를 만드는 방법보다 훨씬 편리해 보인다.

로우를 표현하는 클래스를 정의할 때는 각 클래스 속성에 대응하는 칼럼의 이름을 지정해야 한다.

In [5]:
class Customer(object):
    # 클래스 속성
    first_name = Field('first_name')
    last_name = Field('Last_name')
    prefix = Field('prefix')
    suffix = Field('suffix')

클래스를 사용하는 방법은 간단하다. 다음 코드에서는 Field descriptor가 인스턴스 딕셔너리 `__dict__`를 기대한 대로 수정한 방식을 볼 수 있다.

In [6]:
foo = Customer()
print('Before:', repr(foo.first_name), foo.__dict__)
foo.first_name = 'Euclid'
print('After: ', repr(foo.first_name), foo.__dict__)

Before: '' {}
After:  'Euclid' {'_first_name': 'Euclid'}


하지만 중복되는 것처럼 보인다. 이미 class 문 본문에서 Field 객체를 생성하여 `Customer.first_name`에 할당할 때 필드의 이름을 선언했다.

문제는 Customer 클래스 정의에서 연산 순서가 왼쪽에서 오른쪽으로 읽는 방식과 반대라는 점이다. 먼저 Field 생성자는 `Field('first_name')` 형태로 호출한다. 다음으로 호출의 반환 값을 `Customer.field_name` 에게 할당한다. 그러므로 Field에서는 자신이 어떤 클래스 속성을 할당될지 미리 알 방법이 없다.

중복성을 제거하려면 메타클래스를 사용하면 된다. 메타클래스를 이용하면 class 문을 직접 후킹하여 class 본문이 끝나자마자 원하는 동작을 처리할 수 있다.

In [13]:
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        for key, value in class_dict.items():
            if isinstance(value, Field):
                value.name = key
                value.internal_name = '_' + key
        cls = type.__new__(meta, name, bases, class_dict)
        return cls

다음은 메타클래스를 사용하는 기반 클래스를 정의한 코드다. 데이터베이스 로우를 표현하는 클래스가 모두 이 클래스를 상속하게 해서 모두 메타클래스를 사용하게 해야한다.

In [14]:
class DatabaseRow(object, metaclass=Meta):
    pass

메타클래스를 사용하게 해도 field descriptor는 변경이 거의 없다. 유일한 차이는 더는 생성자에 인수를 넘길 필요가 없다는 점이다. 대신 필드 디스크립터의 속성은 위의 `Meta.__new__`메서드로 설정된다.

In [17]:
class Field(object):
    def __init__(self):
        # 메타클래스가 이 속성들을 할당함
        self.name = None
        self.internal_name = None
    
    def __get__(self, instance, instance_type):
        if instance is None: return self
        return getattr(instance, self.internal_name, '')
    
    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)
    
class BetterCustomer(DatabaseRow):
    first_name = Field()
    last_name = Field()
    prefix = Field()
    suffix = Field()

In [18]:
foo = BetterCustomer()
print('Before:', repr(foo.first_name), foo.__dict__)
foo.first_name = 'Euler'
print('After: ', repr(foo.first_name), foo.__dict__)

Before: '' {}
After:  'Euler' {'_first_name': 'Euler'}
